In [ ]:
%%writefile phonebook.cu

#include <bits/stdc++.h>
using namespace std;
#include <cuda.h>

struct Contact{
    char name[65];
    char phone_number[65];
};

string getInput(ifstream& file){
    string ans;
    char c;
    int readSuru = 0;
    while(file.get(c)){
        if(c == '\"'){
            if(readSuru == 1) break;
            readSuru = 1;
        }else{
            if(readSuru){
                ans.push_back(c);
            }
        }
    }
    return ans;
}

__device__ bool check(const char* str1, const char* str2){
    for(int i = 0; str1[i] != '\0'; i++){
        int flag = 1;
        for(int j = 0; str2[j] != '\0' ; j++){
            if(str1[i + j] != str2[j] || str1[i + j] == '\0'){
                flag = 0;
                break;
            }
        }
        if(flag == 1) return true;
    }
    return false;
}

__device__ void my_strcpy(char* dest, const char* src) {
    int i = 0;
    while (src[i] != '\0' && i < 64) {  // ensure we don't overflow 65 bytes
        dest[i] = src[i];
        i++;
    }
    dest[i] = '\0';
}

__global__ void myKernel(Contact* phoneBook, const char* pat, int totalContacts, char* GPUphoneBookName, char* GPUphoneNumber, bool* foundFlag){
    int globalThreadId = blockIdx.x * blockDim.x + threadIdx.x;
    if(globalThreadId < totalContacts){
        if(check(phoneBook[globalThreadId].name, pat)){
            // Use manual copy (my_strcpy) instead of strcpy
            my_strcpy(GPUphoneBookName, phoneBook[globalThreadId].name);
            my_strcpy(GPUphoneNumber, phoneBook[globalThreadId].phone_number);
            *foundFlag = true;
        }
    }
}


int main(int argc, char* argv[])
{
    /*if(argc < 3){
        cerr << "Usage: ./phonebook_search <search_name> <threads_per_block>\n";
        return 1;
    }*/

    int threadsPerBlock = atoi(argv[2]);

    ifstream myfile("/content/drive/MyDrive/CUDALabFiles/contact_list.txt");
    vector<Contact> phoneBook;

    int count = 0;

    while(myfile.peek() != EOF){
        if(count > 5000000) break;
        count++;

        string name = getInput(myfile);
        string phoneNum = getInput(myfile);

        Contact c;
        strcpy(c.name, name.c_str());
        strcpy(c.phone_number, phoneNum.c_str());

        phoneBook.push_back(c);
    }

    string search_name = argv[1];
    char pat[65];
    strcpy(pat, search_name.c_str());

    // Copy pattern to device
    char* d_pat;
    cudaMalloc(&d_pat, 65);
    cudaMemcpy(d_pat, pat, 65, cudaMemcpyHostToDevice);

    int n = phoneBook.size();
    Contact* d_phoneBook;
    cudaMalloc(&d_phoneBook, n * sizeof(Contact));
    cudaMemcpy(d_phoneBook, phoneBook.data(), n * sizeof(Contact), cudaMemcpyHostToDevice);

    // Allocate device memory for result name/number + flag
    char* GPUphoneBookName;
    char* GPUphoneNumber;
    bool* GPUfoundFlag;
    cudaMalloc(&GPUphoneBookName, 65);
    cudaMalloc(&GPUphoneNumber, 65);
    cudaMalloc(&GPUfoundFlag, sizeof(bool));

    // Initialize GPUfoundFlag = false
    bool initFlag = false;
    cudaMemcpy(GPUfoundFlag, &initFlag, sizeof(bool), cudaMemcpyHostToDevice);


    // Launching kernel
    int blocks = (n + threadsPerBlock - 1) / threadsPerBlock;
    myKernel <<<blocks, threadsPerBlock>>> (d_phoneBook, d_pat, n, GPUphoneBookName, GPUphoneNumber, GPUfoundFlag);

    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        fprintf(stderr, "Kernel launch failed: %s\n", cudaGetErrorString(err));
        return 1;
    }

    cudaDeviceSynchronize();

    // Copy back result and flag
    char CPUphoneBookName[65], CPUphoneNumber[65];
    bool foundFlag;
    cudaMemcpy(&foundFlag, GPUfoundFlag, sizeof(bool), cudaMemcpyDeviceToHost);

    if(foundFlag){
        cudaMemcpy(CPUphoneBookName, GPUphoneBookName, 65, cudaMemcpyDeviceToHost);
        cudaMemcpy(CPUphoneNumber, GPUphoneNumber, 65, cudaMemcpyDeviceToHost);
        cout << "Found: " << CPUphoneBookName << " " << CPUphoneNumber << endl;
    } else {
        cout << "No matching contacts found." << endl;
    }

    // Cleanup
    cudaFree(d_pat);
    cudaFree(d_phoneBook);
    cudaFree(GPUphoneBookName);
    cudaFree(GPUphoneNumber);
    cudaFree(GPUfoundFlag);

    return 0;
}


In [ ]:
!nvcc -arch=sm_75 phonebook.cu -o pb

In [ ]:
!time ./pb Bob 10 > output.txt